The notebook aims to highlight **`the ease of using autoML on containers to solve a common classification problem`**
- Loads data to CAS
- Runs AutoML (datasciencepilot) on CAS
- Generates the champ model
- Scores data with champ model
- Saves scoring results & model to the filesystem
- Spawns h2o session
- Runs h2o automl against the data
- Generates the champ model
- Optionally,saves the champ model 

In [1]:
import swat
from _config import container_cas35_login
from swat.render import render_html
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from matplotlib import pyplot as plt
from IPython.display import HTML
%matplotlib inline
# from plotnine import *
import os, time
#os.environ['CAS_CLIENT_SSL_CA_LIST']=r'/ext_str/beast_trustfiles/nBeast_trustfiles/trustedcerts.pem'
from IPython.display import IFrame
%reload_ext autoreload
%autoreload 2

In [2]:
host = container_cas35_login()[2]
user = container_cas35_login()[0]
pswd = container_cas35_login()[1]

In [3]:
sess = swat.CAS(host,5571,user,pswd)
sess.setsessopt(caslib="casuser")
sess.setsessopt(locale='en-US')
sess.loadactionset(actionset="dataSciencePilot")

NOTE: 'CASUSER(sasdemo)' is now the active caslib.
NOTE: Added action set 'dataSciencePilot'.


[actionset]

 'dataSciencePilot'

+ Elapsed: 0.00539s, user: 0.00162s, sys: 0.00367s, mem: 0.22mb

In [4]:
sess.help(actionset="dataSciencePilot")["dataSciencePilot"]

NOTE: Information for action set 'dataSciencePilot':
NOTE:    dataSciencePilot
NOTE:       exploreData - Exploration, automatic variable analysis and grouping using comprehensive statistical profiling of the variables.
NOTE:       screenVariables - Screens noise variables and variables that need special transformations to be useful in the downstream analytics.
NOTE:       analyzeMissingPatterns - Missing pattern analysis
NOTE:       exploreCorrelation - Explore linear and nonlinear correlation among the variables.
NOTE:       detectInteractions - Variable interaction detection and ranking
NOTE:       generateShadowFeatures - Generate shadow features.
NOTE:       featureMachine - Automated feature transformation and generation engine
NOTE:       selectFeatures - Feature selection
NOTE:       dsAutoMl - Automated machine learning pipeline exploration, execution and ranking.


,name,description
0,exploreData,"Exploration, automatic variable analysis and g..."
1,screenVariables,Screens noise variables and variables that nee...
2,analyzeMissingPatterns,Missing pattern analysis
3,exploreCorrelation,Explore linear and nonlinear correlation among...
4,detectInteractions,Variable interaction detection and ranking
5,generateShadowFeatures,Generate shadow features.
6,featureMachine,Automated feature transformation and generatio...
7,selectFeatures,Feature selection
8,dsAutoMl,Automated machine learning pipeline exploratio...


In [5]:
churn_df=pd.read_csv('churn-vx.csv')
churn_df.columns = [i.replace(' ','_').replace("'",'').lower() for i in churn_df.columns]
#Check out how the resultant dataset look like
churn_df.head()

,account_length,vmail_message,day_mins,eve_mins,night_mins,intl_mins,custserv_calls,churn,intl_plan,vmail_plan,...,eve_charge,night_calls,night_charge,intl_calls,intl_charge,state,area_code,phone,total_mins,total_charge
0,139,0,271.60,156.00,136.30,11.60,2,0,0,0,...,13.26,108,6.13,9,3.13,WV,415,403-9766,575.50,68.69
1,93,0,190.70,218.20,129.60,8.10,3,0,0,0,...,18.55,121,5.83,3,2.19,VT,510,386-2923,546.60,58.99
2,147,24,219.90,208.50,352.50,8.10,3,0,1,1,...,17.72,111,15.86,4,2.19,OH,415,365-5682,789.00,73.15
3,80,0,105.80,43.90,189.60,13.10,0,0,0,0,...,3.73,87,8.53,5,3.54,MN,415,332-2137,352.40,33.79
4,79,0,157.60,194.10,231.50,9.40,5,1,0,0,...,16.50,86,10.42,10,2.54,CA,510,416-8701,592.60,56.25


In [6]:
# Load Table to CAS
out=sess.upload(churn_df,casout=dict(name='churn',caslib='casuser'))
sess.loadactionset('fedSQL') #Enable SQL actions - for distributed SQL         
out=out.casTable #get CASTAble
#programmatically build query
col_list= [i for i in out.columns if i not in ('area_code','churn','intl_plan','vmail_plan')]
cas_lib='casuser'
option_params='{options replace=true}'
query = """create table {}.churn {} as select {}, 
cast(intl_plan as char) as intl_plan,
cast(vmail_plan as char) as vmail_plan,
cast(area_code as varchar) as area_code,
cast(churn as varchar) as churn
from casuser.churn """.format(cas_lib,
                              option_params,
                              col_list).replace('[','').replace(']','').replace("'",'')

#execute query and check the results
sess.fedsql.execdirect(query) # run the query
out = sess.CASTable('CHURN', caslib ='casuser') #get the results
render_html(out.fetch(to=5)) #view the results

NOTE: Cloud Analytic Services made the uploaded file available as table CHURN in caslib CASUSER(sasdemo).
NOTE: The table CHURN has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
NOTE: Added action set 'fedSQL'.
NOTE: Table CHURN was created in caslib CASUSER(sasdemo) with 3500 rows returned.


Selected Rows from Table CHURN 
 account_length 
 vmail_message 
 day_mins 
 eve_mins 
 night_mins 
 intl_mins 
 custserv_calls 
 day_calls 
 day_charge 
 eve_calls 
 eve_charge 
 night_calls 
 night_charge 
 intl_calls 
 intl_charge 
 state 
 phone 
 total_mins 
 total_charge 
 INTL_PLAN 
 VMAIL_PLAN 
 AREA_CODE 
 CHURN 
 
 
 139 0 271.6 156 136.3 11.6 2 130 46.17 131 13.26 108 6.13 9 3.13 WV 403-9766 575.5 68.69 0 0 415 0 
 93 0 190.7 218.2 129.6 8.1 3 114 32.42 111 18.55 121 5.83 3 2.19 VT 386-2923 546.6 58.99 0 0 510 0 
 147 24 219.9 208.5 352.5 8.1 3 118 37.38 116 17.72 111 15.86 4 2.19 OH 365-5682 789 73.15 1 1 415 0 
 80 0 105.8 43.9 189.6 13.1 0 110 17.99 88 3.73 87 8.53 5 3.54 MN 332-2137 352.4 33.79 0 0 415 0 
 79 0 157.6 194.1 231.5 9.4 5 85 26.79 92 16.5 86 10.42 10 2.54 CA 416-8701 592.6 56.25 0 0 510 1

In [7]:
effect_vars={'area_code','account_length', 'custserv_calls', 'day_calls', 'day_charge', 
             'day_mins', 'eve_calls','eve_charge', 'eve_mins', 'intl_calls', 
             'intl_charge', 'intl_mins', 'intl_plan','night_calls','night_charge',
             'night_mins', 'vmail_message', 'vmail_plan'}

In [8]:
out.columninfo()

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,account_length,,1,double,8,12,,0,0
1,vmail_message,,2,double,8,12,,0,0
2,day_mins,,3,double,8,12,,0,0
3,eve_mins,,4,double,8,12,,0,0
4,night_mins,,5,double,8,12,,0,0
5,intl_mins,,6,double,8,12,,0,0
6,custserv_calls,,7,double,8,12,,0,0
7,day_calls,,8,double,8,12,,0,0
8,day_charge,,9,double,8,12,,0,0
9,eve_calls,,10,double,8,12,,0,0


In [9]:
sess.dataSciencePilot.dsAutoMl(
#     debugLevel = 1,
        table                   = out,
        target                  = "CHURN",
        inputs = effect_vars,
#         explorationPolicy       = {},
#         screenPolicy            = {},
#         selectionPolicy         = {},
        transformationPolicy    = {"missing":True, "cardinality":True,
                                   "entropy":True, "iqv":True,
                                   "skewness":True, "kurtosis":True, "Outlier":True},
        modelTypes              = ["decisionTree", "GRADBOOST"],
        objective               = "AUC",
        sampleSize              = 20,
        topKPipelines           = 10,
        kFolds                  = 2,
        transformationOut       = {"name" : "TRANSFORMATION_OUT", "replace" : True},
        featureOut              = {"name" : "FEATURE_OUT", "replace" : True},
        pipelineOut             = {"name" : "PIPELINE_OUT", "replace" : True},
        saveState               = dict(modelNamePrefix='churn_model', replace = True)      
    )

NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added action set 'decisionTree'.
NOTE: Early stopping is activated; 'NTREE' will not be tuned.
NOTE: Added action set 'autotune'.
NOTE: Added a

,Descr,Value
0,Number of Tree Nodes,111.00
1,Max Number of Branches,2.00
2,Number of Levels,10.00
3,Number of Leaves,56.00
4,Number of Bins,100.00
5,Minimum Size of Leaves,5.00
6,Maximum Size of Leaves,1568.00
7,Number of Variables,10.00
8,Confidence Level for Pruning,0.25
9,Number of Observations Used,3500.00


In [10]:
sess.tableinfo()

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,CHURN,3500,23,0,utf-8,2020-07-31T22:52:27+00:00,2020-07-31T22:52:27+00:00,2020-07-31T22:52:48+00:00,UTF8,1911855146.80,...,0,0,0,,,0,sasdemo,,,nan
1,CHURN_MODEL_FM_,1,2,0,utf-8,2020-07-31T22:52:49+00:00,2020-07-31T22:52:49+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855168.66,...,0,0,0,,,0,sasdemo,,,nan
2,TRANSFORMATION_OUT,19,21,0,utf-8,2020-07-31T22:52:49+00:00,2020-07-31T22:52:49+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855168.71,...,0,0,0,,,0,sasdemo,,,nan
3,FEATURE_OUT,108,15,0,utf-8,2020-07-31T22:52:49+00:00,2020-07-31T22:52:49+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855168.71,...,0,0,0,,,0,sasdemo,,,nan
4,CHURN_MODEL_GRADBOOST_1,1,2,0,utf-8,2020-07-31T23:00:10+00:00,2020-07-31T23:00:10+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855610.06,...,0,0,0,,,0,sasdemo,,,nan
5,CHURN_MODEL_GRADBOOST_2,1,2,0,utf-8,2020-07-31T23:00:27+00:00,2020-07-31T23:00:27+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855626.83,...,0,0,0,,,0,sasdemo,,,nan
6,CHURN_MODEL_GRADBOOST_3,1,2,0,utf-8,2020-07-31T23:00:48+00:00,2020-07-31T23:00:48+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855647.89,...,0,0,0,,,0,sasdemo,,,nan
7,CHURN_MODEL_GRADBOOST_4,1,2,0,utf-8,2020-07-31T23:01:11+00:00,2020-07-31T23:01:11+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855670.74,...,0,0,0,,,0,sasdemo,,,nan
8,CHURN_MODEL_GRADBOOST_5,1,2,0,utf-8,2020-07-31T23:01:28+00:00,2020-07-31T23:01:28+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855687.84,...,0,0,0,,,0,sasdemo,,,nan
9,PIPELINE_OUT,10,51,0,utf-8,2020-07-31T23:01:28+00:00,2020-07-31T23:01:28+00:00,2020-07-31T23:01:28+00:00,UTF8,1911855687.87,...,0,0,0,,,0,sasdemo,,,nan


In [11]:
churn_model_fm_ = sess.CASTable('CHURN_MODEL_FM_')

In [12]:
sess.loadactionset('astore')

NOTE: Added action set 'astore'.


[actionset]

 'astore'

+ Elapsed: 0.000734s, user: 0.000647s, mem: 0.203mb

In [13]:
sess.describe(churn_model_fm_)

[Key]

 Store Key
 
                                         Key
 0  C9CECE66B8B85B460A776A4B4778E63DBF7D1F2A

[Description]

 Basic Information
 
          Attribute               Value
 0  Analytic Engine                 fte
 1     Time Created  31Jul2020:22:52:48

[InputVariables]

 Input Variables
 
               Name  Length   Role      Type    RawType FormatName
 0       day_charge    8.00  Input  Interval        Num           
 1         day_mins    8.00  Input  Interval        Num           
 2       eve_charge    8.00  Input  Interval        Num           
 3         eve_mins    8.00  Input  Interval        Num           
 4      intl_charge    8.00  Input  Interval        Num           
 5        intl_mins    8.00  Input  Interval        Num           
 6     night_charge    8.00  Input  Interval        Num           
 7       night_mins    8.00  Input  Interval        Num           
 8   account_length    8.00  Input  Interval        Num           
 9   custserv_calls    8.00  Input  Interval        Num           
 10       day_calls    8.00  Input  Interval        Num           
 11       eve_calls    8.00  Input  Interval        Num           
 12      intl_calls    8.00  Input  Interval        Num           
 13     night_calls    8.00  Input  Interval        Num           
 14   vmail_message    8.00  Input  Interval        Num           
 15       AREA_CODE    3.00  Input  Interval  Character           
 16       INTL_PLAN    1.00  Input  Interval  Character           
 17      VMAIL_PLAN    1.00  Input  Interval  Character           

[OutputVariables]

 Output Variables
 
                                 Name  Length Type  \
 0    nhoks_nloks_dtree_10_day_charge    8.00  Num   
 1     nhoks_nloks_dtree_5_day_charge    8.00  Num   
 2    nhoks_nloks_pow_n0_5_day_charge    8.00  Num   
 3      nhoks_nloks_pow_p2_day_charge    8.00  Num   
 4      nhoks_nloks_dtree_10_day_mins    8.00  Num   
 5       nhoks_nloks_dtree_5_day_mins    8.00  Num   
 6           nhoks_nloks_log_day_mins    8.00  Num   
 7      nhoks_nloks_pow_p0_5_day_mins    8.00  Num   
 8         cpy_int_med_imp_eve_charge    8.00  Num   
 9    nhoks_nloks_dtree_10_eve_charge    8.00  Num   
 10    nhoks_nloks_dtree_5_eve_charge    8.00  Num   
 11     nhoks_nloks_pow_p1_eve_charge    8.00  Num   
 12     nhoks_nloks_pow_p2_eve_charge    8.00  Num   
 13          cpy_int_med_imp_eve_mins    8.00  Num   
 14     nhoks_nloks_dtree_10_eve_mins    8.00  Num   
 15      nhoks_nloks_dtree_5_eve_mins    8.00  Num   
 16       nhoks_nloks_pow_p1_eve_mins    8.00  Num   
 17  nhoks_nloks_dtree_10_intl_charge    8.00  Num   
 18   nhoks_nloks_dtree_5_intl_charge    8.00  Num   
 19       nhoks_nloks_log_intl_charge    8.00  Num   
 20    nhoks_nloks_pow_p2_intl_charge    8.00  Num   
 21         cpy_int_med_imp_intl_mins    8.00  Num   
 22    nhoks_nloks_dtree_10_intl_mins    8.00  Num   
 23     nhoks_nloks_dtree_5_intl_mins    8.00  Num   
 24      nhoks_nloks_pow_p1_intl_mins    8.00  Num   
 25      nhoks_nloks_pow_p2_intl_mins    8.00  Num   
 26      cpy_int_med_imp_night_charge    8.00  Num   
 27       nhoks_nloks_dtree_10_var_2_    8.00  Num   
 28  nhoks_nloks_dtree_5_night_charge    8.00  Num   
 29   nhoks_nloks_pow_n2_night_charge    8.00  Num   
 30   nhoks_nloks_pow_p1_night_charge    8.00  Num   
 31        cpy_int_med_imp_night_mins    8.00  Num   
 32   nhoks_nloks_dtree_10_night_mins    8.00  Num   
 33    nhoks_nloks_dtree_5_night_mins    8.00  Num   
 34     nhoks_nloks_pow_p1_night_mins    8.00  Num   
 35         hc_lbl_cnt_account_length    8.00  Num   
 36     hc_tar_frq_rat_account_length    8.00  Num   
 37       cpy_nom_mode_imp_lab_var_1_    8.00  Num   
 38         lchehi_lab_custserv_calls    8.00  Num   
 39              hc_lbl_cnt_day_calls    8.00  Num   
 40          hc_tar_frq_rat_day_calls    8.00  Num   
 41              hc_lbl_cnt_eve_calls    8.00  Num   
 42          hc_tar_frq_rat_eve_calls    8.00  Num   
 43   

In [14]:
sess.score(table='CHURN',casout=dict(name='fm_result', caslib='casuser', replace =True),rstore=churn_model_fm_, copyvars=['CHURN'])

[OutputCasTables]

              casLib       Name  Rows  Columns  \
 0  CASUSER(sasdemo)  fm_result  3500       54   
 
                                            casTable  
 0  CASTable('fm_result', caslib='CASUSER(sasdemo)')  

[Timing]

 Task Timing
 
                  Task  Seconds  Percent
 0   Loading the Store     0.00     0.01
 1  Creating the State     0.01     0.37
 2             Scoring     0.01     0.61
 3               Total     0.01     1.00

+ Elapsed: 0.0183s, user: 0.0387s, sys: 0.00317s, mem: 57.6mb

In [15]:
fm_result = sess.CASTable('fm_result')

In [16]:
fm_result.head()

,nhoks_nloks_dtree_10_day_charge,nhoks_nloks_dtree_5_day_charge,nhoks_nloks_pow_n0_5_day_charge,nhoks_nloks_pow_p2_day_charge,nhoks_nloks_dtree_10_day_mins,nhoks_nloks_dtree_5_day_mins,nhoks_nloks_log_day_mins,nhoks_nloks_pow_p0_5_day_mins,cpy_int_med_imp_eve_charge,nhoks_nloks_dtree_10_eve_charge,...,lchehi_lab_intl_calls,hc_lbl_cnt_night_calls,hc_tar_frq_rat_night_calls,lcnhenhi_dtree10_vmail_message,lcnhenhi_dtree5_vmail_message,cpy_nom_mode_imp_lab_AREA_CODE,lchehi_lab_AREA_CODE,cpy_nom_mode_imp_lab_INTL_PLAN,cpy_nom_mode_imp_lab_VMAIL_PLAN,CHURN
0,10.00,5.00,0.15,2225.01,10.00,5.00,5.61,16.51,13.26,3.00,...,10.00,19.00,0.90,7.00,4.00,2.00,2.00,1.00,1.00,0
1,7.00,2.00,0.17,1116.90,7.00,2.00,5.26,13.85,18.55,7.00,...,4.00,45.00,0.82,7.00,4.00,3.00,3.00,1.00,1.00,0
2,7.00,2.00,0.16,1473.02,7.00,2.00,5.40,14.86,17.72,7.00,...,5.00,27.00,0.78,2.00,2.00,2.00,2.00,2.00,2.00,0
3,2.00,2.00,0.23,360.62,2.00,2.00,4.67,10.33,3.73,1.00,...,6.00,16.00,0.90,7.00,4.00,2.00,2.00,1.00,1.00,0
4,5.00,2.00,0.19,772.28,5.00,2.00,5.07,12.59,16.50,6.00,...,11.00,29.00,0.87,7.00,4.00,3.00,3.00,1.00,1.00,1


#### Save Feature Machine Astore to the Disk

In [17]:
sess.save(table=churn_model_fm_,name='automl_churn_features_astore.sashdat', replace=True)

NOTE: Cloud Analytic Services saved the file automl_churn_features_astore.sashdat in caslib CASUSER(sasdemo).


[caslib]

 'CASUSER(sasdemo)'

[name]

 'automl_churn_features_astore.sashdat'

+ Elapsed: 0.00491s, user: 0.000846s, sys: 0.00204s, mem: 16.5mb

#### Save Top N PipeLines to a CSV File

In [ ]:
pipeline_out = sess.CASTable('PIPELINE_OUT')

pipeline_out.iloc[:,:'NFeatures'].to_frame().to_csv('topNPipeLines.csv', index=False)

#### Score with Champion

In [ ]:
champ_model = sess.CASTable('CHURN_MODEL_GRADBOOST_1')

sess.score(table=fm_result,
           casout=dict(name='model_result', caslib='casuser', replace =True),
           rstore=champ_model)

#### Save Champion Model

In [ ]:
sess.save(table=champ_model,name='automl_churn_champ.sashdat', replace=True)

In [ ]:
result = sess.CASTable('model_result')
result.head()

#### Save Scored Results

In [ ]:
sess.save(table=result,name='model_scoring_result.sashdat', replace=True)

In [ ]:
sess.fileinfo(allfiles=True)

In [ ]:
sess.close()

### Now we can run the h2o autoML

In [18]:
import h2o
from h2o.automl import H2OAutoML
h2o.init('http://ccbu-vidk.dlviyacluster.sashq-d.openstack.sas.com:54321/')

Checking whether there is an H2O instance running at http://ccbu-vidk.dlviyacluster.sashq-d.openstack.sas.com:54321 . connected.


H2O_cluster_uptime:,17 mins 07 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,3 months and 2 days
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,744 Mb
H2O_cluster_total_cores:,24
H2O_cluster_allowed_cores:,24
H2O_cluster_status:,"accepting new members, healthy"


In [24]:
churn_h2o = h2o.H2OFrame(out.to_frame())

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [25]:
churn_h2o.head()

account_length,vmail_message,day_mins,eve_mins,night_mins,intl_mins,custserv_calls,day_calls,day_charge,eve_calls,eve_charge,night_calls,night_charge,intl_calls,intl_charge,state,phone,total_mins,total_charge,INTL_PLAN,VMAIL_PLAN,AREA_CODE,CHURN
139,0,271.6,156,136.3,11.6,2,130,46.17,131,13.26,108,6.13,9,3.13,WV,403-9766,575.5,68.69,0,0,415,0
93,0,190.7,218.2,129.6,8.1,3,114,32.42,111,18.55,121,5.83,3,2.19,VT,386-2923,546.6,58.99,0,0,510,0
147,24,219.9,208.5,352.5,8.1,3,118,37.38,116,17.72,111,15.86,4,2.19,OH,365-5682,789,73.15,1,1,415,0
80,0,105.8,43.9,189.6,13.1,0,110,17.99,88,3.73,87,8.53,5,3.54,MN,332-2137,352.4,33.79,0,0,415,0
79,0,157.6,194.1,231.5,9.4,5,85,26.79,92,16.5,86,10.42,10,2.54,CA,416-8701,592.6,56.25,0,0,510,1
112,30,60.6,165.9,132.8,13.3,0,113,10.3,96,14.1,99,5.98,7,3.59,NY,396-7687,372.6,33.97,0,1,408,0
71,0,258.4,126.8,182.4,9.7,4,132,43.93,119,10.78,87,8.21,8,2.62,GA,391-7166,577.3,65.54,0,0,415,0
101,28,105.9,231.7,281.3,10.7,1,132,18,107,19.69,120,12.66,5,2.89,ND,379-4583,629.6,53.24,0,1,415,0
99,0,159.7,155.4,255.7,8.4,1,83,27.15,121,13.21,114,11.51,3,2.27,ID,354-7025,579.2,54.14,0,0,408,0
99,0,142.3,204.5,203.1,9.1,0,89,24.19,95,17.38,114,9.14,1,2.46,LA,379-9821,559,53.17,0,0,510,0


In [26]:
x = churn_h2o.columns
y = 'CHURN'
x.remove(y)
x.remove('AREA_CODE')

In [27]:
churn_h2o[y] = churn_h2o[y].asfactor()

In [28]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1,stopping_metric='auc',sort_metric='auc')
aml.train(x=x, y=y, training_frame=churn_h2o)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XRT_1_AutoML_20200604_175436,0.977046,0.125733,0.96385,0.0326112,0.146882,0.0215744
XGBoost_3_AutoML_20200604_175436,0.975175,0.0756336,0.959001,0.0450463,0.12531,0.0157026
DRF_1_AutoML_20200604_175436,0.974992,0.192309,0.964276,0.0318206,0.127707,0.016309
XGBoost_grid__1_AutoML_20200604_175436_model_2,0.972676,0.0671988,0.963507,0.0283899,0.0978064,0.00956609
StackedEnsemble_BestOfFamily_AutoML_20200604_175436,0.972194,0.0510459,0.96237,0.0275226,0.0955173,0.00912355
XGBoost_grid__1_AutoML_20200604_175436_model_4,0.971928,0.0792362,0.95386,0.0526476,0.12781,0.0163355
GBM_grid__1_AutoML_20200604_175436_model_2,0.971297,0.114206,0.958076,0.0396768,0.161408,0.0260527
GBM_1_AutoML_20200604_175436,0.969703,0.0781223,0.953468,0.0335168,0.124067,0.0153927
GBM_4_AutoML_20200604_175436,0.968618,0.0708647,0.958428,0.0307876,0.116138,0.0134879
XGBoost_grid__1_AutoML_20200604_175436_model_1,0.967744,0.0866015,0.949752,0.0478139,0.132959,0.0176782


In [162]:
h2o.shutdown()

/home/centos/.conda/envs/dl4viya/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  """Entry point for launching an IPython kernel.


H2O session _sid_92b0 closed.
